In [13]:
import polars as pol
import pandas as pd
import pickle
import tqdm
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [14]:
test_df_main = pol.read_parquet("./data/test.parquet")
test_df_main.head()

# read model from file
models = []
for i in range(5):
    with open(f'lgbm_fold{i}.pkl', 'rb') as f:
        models.append(pickle.load(f))

# read features from file
with open('features.pkl', 'rb') as f:
    features = pickle.load(f)


In [15]:
models[1]

LGBMClassifier(colsample_bytree=0.4532763195519286, learning_rate=0.002,
               max_depth=8, metric='binary_logloss', min_child_samples=89,
               min_split_gain=0.09436691418409165, num_iterations=3000,
               num_leaves=78, objective='binary', random_seed=123,
               reg_alpha=3.255790866684656, reg_lambda=1.6509939612937656,
               subsample=0.36636553619171086)

In [16]:
DRUG_TYPES = [
 'DRUG_TYPE_0', 'DRUG_TYPE_1', 'DRUG_TYPE_2', 'DRUG_TYPE_3', 'DRUG_TYPE_4', 'DRUG_TYPE_5', 'DRUG_TYPE_6',
 'DRUG_TYPE_7', 'DRUG_TYPE_8', 'DRUG_TYPE_9', 'DRUG_TYPE_10', 'DRUG_TYPE_11', 'DRUG_TYPE_12', 'DRUG_TYPE_13',
 'DRUG_TYPE_14', 'DRUG_TYPE_15', 'DRUG_TYPE_16', 'DRUG_TYPE_17', 'DRUG_TYPE_18',
]
SYMPTOM_TYPES = ['SYMPTOM_TYPE_0', 'SYMPTOM_TYPE_1', 'SYMPTOM_TYPE_2','SYMPTOM_TYPE_3', 'SYMPTOM_TYPE_4', 'SYMPTOM_TYPE_5', 'SYMPTOM_TYPE_6',
 'SYMPTOM_TYPE_7', 'SYMPTOM_TYPE_8', 'SYMPTOM_TYPE_9', 'SYMPTOM_TYPE_10', 'SYMPTOM_TYPE_11', 'SYMPTOM_TYPE_12', 'SYMPTOM_TYPE_13',
 'SYMPTOM_TYPE_14', 'SYMPTOM_TYPE_15', 'SYMPTOM_TYPE_16', 'SYMPTOM_TYPE_17', 'SYMPTOM_TYPE_18', 'SYMPTOM_TYPE_19',
 'SYMPTOM_TYPE_20', 'SYMPTOM_TYPE_21', 'SYMPTOM_TYPE_22', 'SYMPTOM_TYPE_23', 'SYMPTOM_TYPE_24', 'SYMPTOM_TYPE_25', 'SYMPTOM_TYPE_26',
 'SYMPTOM_TYPE_27', 'SYMPTOM_TYPE_28', 'SYMPTOM_TYPE_29',
 ]
TEST_TYPES = ['TEST_TYPE_0', 'TEST_TYPE_1', 'TEST_TYPE_2', 'TEST_TYPE_3', 'TEST_TYPE_4', 'TEST_TYPE_5',]
TARGET = 'TARGET DRUG'
PRIMARY = 'PRIMARY_DIAGNOSIS'

variables = [PRIMARY] + SYMPTOM_TYPES +  TEST_TYPES + DRUG_TYPES  +  [TARGET]
varDict = {}
count=1
for i in variables:
    varDict[i]=count
    count+=1

print(varDict)
print(f"DRUG_TYPES: {len(DRUG_TYPES)}")
print(f"SYMPTOM_TYPES: {len(SYMPTOM_TYPES)}")
print(f"TEST_TYPES: {len(TEST_TYPES)}")



{'PRIMARY_DIAGNOSIS': 1, 'SYMPTOM_TYPE_0': 2, 'SYMPTOM_TYPE_1': 3, 'SYMPTOM_TYPE_2': 4, 'SYMPTOM_TYPE_3': 5, 'SYMPTOM_TYPE_4': 6, 'SYMPTOM_TYPE_5': 7, 'SYMPTOM_TYPE_6': 8, 'SYMPTOM_TYPE_7': 9, 'SYMPTOM_TYPE_8': 10, 'SYMPTOM_TYPE_9': 11, 'SYMPTOM_TYPE_10': 12, 'SYMPTOM_TYPE_11': 13, 'SYMPTOM_TYPE_12': 14, 'SYMPTOM_TYPE_13': 15, 'SYMPTOM_TYPE_14': 16, 'SYMPTOM_TYPE_15': 17, 'SYMPTOM_TYPE_16': 18, 'SYMPTOM_TYPE_17': 19, 'SYMPTOM_TYPE_18': 20, 'SYMPTOM_TYPE_19': 21, 'SYMPTOM_TYPE_20': 22, 'SYMPTOM_TYPE_21': 23, 'SYMPTOM_TYPE_22': 24, 'SYMPTOM_TYPE_23': 25, 'SYMPTOM_TYPE_24': 26, 'SYMPTOM_TYPE_25': 27, 'SYMPTOM_TYPE_26': 28, 'SYMPTOM_TYPE_27': 29, 'SYMPTOM_TYPE_28': 30, 'SYMPTOM_TYPE_29': 31, 'TEST_TYPE_0': 32, 'TEST_TYPE_1': 33, 'TEST_TYPE_2': 34, 'TEST_TYPE_3': 35, 'TEST_TYPE_4': 36, 'TEST_TYPE_5': 37, 'DRUG_TYPE_0': 38, 'DRUG_TYPE_1': 39, 'DRUG_TYPE_2': 40, 'DRUG_TYPE_3': 41, 'DRUG_TYPE_4': 42, 'DRUG_TYPE_5': 43, 'DRUG_TYPE_6': 44, 'DRUG_TYPE_7': 45, 'DRUG_TYPE_8': 46, 'DRUG_TYPE_9': 47,

In [17]:
test_df1 = test_df_main.with_columns(
    ((pol.col("Date").dt.date() - pol.col("Date").min())/(86400*10**9))
    .cast(pol.Int32)
    .sort()
    .over("Patient-Uid")
    .alias("days_ofcourse")
) # days_ofcourse represents the days since the first day of the course

test_df1.drop_in_place("Date")
test_df1.drop_in_place("__index_level_0__")

test_df1 = test_df1.with_columns(
    (pol.col("days_ofcourse") - pol.col("days_ofcourse").shift(1))
    .fill_null(0)
    .cast(pol.Int32)
    .over("Patient-Uid")
    .alias("days_since_last_incident")
)

test_df1 = test_df1.with_columns(
    pol.col("days_since_last_incident").max()
    .over(["Patient-Uid", "days_ofcourse"])
)
    
test_df1.head()

Patient-Uid,Incident,days_ofcourse,days_since_last_incident
str,str,i32,i32
"""a0f9e8a9-1c7c-…","""SYMPTOM_TYPE_0…",0,0
"""a0f9e8a9-1c7c-…","""DRUG_TYPE_0""",168,168
"""a0f9e8a9-1c7c-…","""DRUG_TYPE_2""",168,168
"""a0f9e8a9-1c7c-…","""DRUG_TYPE_1""",168,168
"""a0f9e8a9-1c7c-…","""SYMPTOM_TYPE_0…",168,168


In [18]:
# aggregate the data by patient-uid and if target is present in the incident then add a column of 1 else 0 within the 30 days_since_last_incident
train_aggs = [  
    *[pol.col('Incident')
    .filter(pol.col('Incident') == c)
    .count()
    .cast(pol.Int32)
    .alias(f'COUNT_{c}')
    for lst in [DRUG_TYPES, SYMPTOM_TYPES ,TEST_TYPES] for c in lst],

    pol.col('Incident')
        .filter(pol.col('Incident') == PRIMARY)
        .count()
        .cast(pol.Int32)
        .alias(f'COUNT_{PRIMARY}'),
    
    pol.col('days_since_last_incident').max().alias('days_since_last_incident_max'),
    pol.col('days_since_last_incident').mean().alias('days_since_last_incident_mean'),
    pol.col('days_since_last_incident').std().alias('days_since_last_incident_std'),
    pol.col('days_since_last_incident').where(pol.col('days_since_last_incident') != 0).min().alias('days_since_last_incident_min'),
    
    pol.col('days_ofcourse').max().alias('days_ofcourse_max'),

    *[pol.col('days_since_last_incident')
    .filter(pol.col('Incident') == c)
    .std().alias(f'days_since_last_incident_{c}_std')
    .fill_null(0)
    for lst in [DRUG_TYPES, SYMPTOM_TYPES, TEST_TYPES] for c in lst], 

    *[pol.col('days_since_last_incident')
    .filter(pol.col('Incident') == c)
    .mean().alias(f'days_since_last_incident_{c}_mean')
    .fill_null(0)
    for lst in [DRUG_TYPES, SYMPTOM_TYPES, TEST_TYPES] for c in lst], 
    
    *[pol.col('days_since_last_incident')
    .filter(pol.col('Incident') == c)
    .max().alias(f'days_since_last_incident_{c}_max')
    .fill_null(0)
    for lst in [DRUG_TYPES, SYMPTOM_TYPES, TEST_TYPES] for c in lst], 

    *[pol.col('days_since_last_incident')
    .filter(pol.col('Incident') == c)
    .min().alias(f'''days_since_last_incident_{c}_min''')
    .fill_null(0)
    for lst in [DRUG_TYPES, SYMPTOM_TYPES, TEST_TYPES] for c in lst], 

    # quantiles 
    *[pol.col('days_since_last_incident')
    .filter(pol.col('Incident') == c)
    .quantile(0.2).alias(f'''days_since_last_incident_{c}_quantile_20''')
    .fill_null(0)
    for lst in [DRUG_TYPES, SYMPTOM_TYPES, TEST_TYPES] for c in lst],

    *[pol.col('days_since_last_incident')
    .filter(pol.col('Incident') == c)
    .quantile(0.4).alias(f'''days_since_last_incident_{c}_quantile_40''')
    .fill_null(0)
    for lst in [DRUG_TYPES, SYMPTOM_TYPES, TEST_TYPES] for c in lst],

    *[pol.col('days_since_last_incident')
    .filter(pol.col('Incident') == c)
    .quantile(0.6).alias(f'''days_since_last_incident_{c}_quantile_60''')
    .fill_null(0)
    for lst in [DRUG_TYPES, SYMPTOM_TYPES, TEST_TYPES] for c in lst],

    *[pol.col('days_since_last_incident')
    .filter(pol.col('Incident') == c)
    .quantile(0.8).alias(f'''days_since_last_incident_{c}_quantile_80''')
    .fill_null(0)
    for lst in [DRUG_TYPES, SYMPTOM_TYPES, TEST_TYPES] for c in lst],
]


test_df1 = test_df1.groupby(["Patient-Uid"], maintain_order=True
                            ).agg(train_aggs
                            ).sort(by="Patient-Uid")

test_df1.head()

Patient-Uid,COUNT_DRUG_TYPE_0,COUNT_DRUG_TYPE_1,COUNT_DRUG_TYPE_2,COUNT_DRUG_TYPE_3,COUNT_DRUG_TYPE_4,COUNT_DRUG_TYPE_5,COUNT_DRUG_TYPE_6,COUNT_DRUG_TYPE_7,COUNT_DRUG_TYPE_8,COUNT_DRUG_TYPE_9,COUNT_DRUG_TYPE_10,COUNT_DRUG_TYPE_11,COUNT_DRUG_TYPE_12,COUNT_DRUG_TYPE_13,COUNT_DRUG_TYPE_14,COUNT_DRUG_TYPE_15,COUNT_DRUG_TYPE_16,COUNT_DRUG_TYPE_17,COUNT_DRUG_TYPE_18,COUNT_SYMPTOM_TYPE_0,COUNT_SYMPTOM_TYPE_1,COUNT_SYMPTOM_TYPE_2,COUNT_SYMPTOM_TYPE_3,COUNT_SYMPTOM_TYPE_4,COUNT_SYMPTOM_TYPE_5,COUNT_SYMPTOM_TYPE_6,COUNT_SYMPTOM_TYPE_7,COUNT_SYMPTOM_TYPE_8,COUNT_SYMPTOM_TYPE_9,COUNT_SYMPTOM_TYPE_10,COUNT_SYMPTOM_TYPE_11,COUNT_SYMPTOM_TYPE_12,COUNT_SYMPTOM_TYPE_13,COUNT_SYMPTOM_TYPE_14,COUNT_SYMPTOM_TYPE_15,COUNT_SYMPTOM_TYPE_16,…,days_since_last_incident_DRUG_TYPE_18_quantile_80,days_since_last_incident_SYMPTOM_TYPE_0_quantile_80,days_since_last_incident_SYMPTOM_TYPE_1_quantile_80,days_since_last_incident_SYMPTOM_TYPE_2_quantile_80,days_since_last_incident_SYMPTOM_TYPE_3_quantile_80,days_since_last_incident_SYMPTOM_TYPE_4_quantile_80,days_since_last_incident_SYMPTOM_TYPE_5_quantile_80,days_since_last_incident_SYMPTOM_TYPE_6_quantile_80,days_since_last_incident_SYMPTOM_TYPE_7_quantile_80,days_since_last_incident_SYMPTOM_TYPE_8_quantile_80,days_since_last_incident_SYMPTOM_TYPE_9_quantile_80,days_since_last_incident_SYMPTOM_TYPE_10_quantile_80,days_since_last_incident_SYMPTOM_TYPE_11_quantile_80,days_since_last_incident_SYMPTOM_TYPE_12_quantile_80,days_since_last_incident_SYMPTOM_TYPE_13_quantile_80,days_since_last_incident_SYMPTOM_TYPE_14_quantile_80,days_since_last_incident_SYMPTOM_TYPE_15_quantile_80,days_since_last_incident_SYMPTOM_TYPE_16_quantile_80,days_since_last_incident_SYMPTOM_TYPE_17_quantile_80,days_since_last_incident_SYMPTOM_TYPE_18_quantile_80,days_since_last_incident_SYMPTOM_TYPE_19_quantile_80,days_since_last_incident_SYMPTOM_TYPE_20_quantile_80,days_since_last_incident_SYMPTOM_TYPE_21_quantile_80,days_since_last_incident_SYMPTOM_TYPE_22_quantile_80,days_since_last_incident_SYMPTOM_TYPE_23_quantile_80,days_since_last_incident_SYMPTOM_TYPE_24_quantile_80,days_since_last_incident_SYMPTOM_TYPE_25_quantile_80,days_since_last_incident_SYMPTOM_TYPE_26_quantile_80,days_since_last_incident_SYMPTOM_TYPE_27_quantile_80,days_since_last_incident_SYMPTOM_TYPE_28_quantile_80,days_since_last_incident_SYMPTOM_TYPE_29_quantile_80,days_since_last_incident_TEST_TYPE_0_quantile_80,days_since_last_incident_TEST_TYPE_1_quantile_80,days_since_last_incident_TEST_TYPE_2_quantile_80,days_since_last_incident_TEST_TYPE_3_quantile_80,days_since_last_incident_TEST_TYPE_4_quantile_80,days_since_last_incident_TEST_TYPE_5_quantile_80
str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""a0f9e8a9-1c7c-…",8,3,2,0,0,1,0,2,16,0,0,1,0,0,0,0,0,0,0,2,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,…,0.0,168.0,0.0,0.0,0.0,0.0,0.0,82.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,168.0,0.0,0.0,0.0,0.0,0.0
"""a0f9e9f9-1c7c-…",2,30,16,0,0,0,47,3,2,5,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35.0,0.0,0.0
"""a0f9ea43-1c7c-…",4,33,10,0,0,0,31,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0
"""a0f9ea7c-1c7c-…",2,0,3,53,0,0,49,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29.0,0.0,0.0,0.0,0.0,0.0
"""a0f9eab1-1c7c-…",5,0,4,0,0,1,41,2,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [19]:
#coverting into a dict having patient_id as key and array of the patients' events with date as values
test_df2 = test_df_main.to_pandas()
history = {}
for _, pid, date, event, _ in tqdm.tqdm(test_df2.itertuples()):
    if pid not in history:
        history[pid]=[[date, event]]
    else:
        history[pid].append([date, event])

#Finding the start date to set that as zero. Subtracting all dates from the start date and sorting it.
for i in tqdm.tqdm(history):
    st_date = history[i][0][0]
    for date, _ in history[i]:
        st_date = min(st_date, date)
    for j in range(len(history[i])):
        history[i][j][0] = (history[i][j][0]-st_date).days
    history[i].sort(key = lambda x: x[0])

arr=[]
for i in tqdm.tqdm(history):
    tmp = [0]*1650
    for j in history[i]:
        tmp.append(varDict[j[1]])
        tmp.pop(0)
    tmp.insert(0, i)
    arr.append(tmp)
arr = pd.DataFrame(arr)

1065524it [00:02, 396208.42it/s]
100%|██████████| 11482/11482 [00:00<00:00, 21277.66it/s]


In [20]:
arr = pd.concat([arr.iloc[:, 0] , arr.iloc[:, -150:]], axis=1)
arr.rename(columns={0:'Patient-Uid'}, inplace=True)

test_df1 = test_df1.to_pandas()
test_df = pd.merge(arr, test_df1, on='Patient-Uid', how='left')
test_df

,Patient-Uid,1501,1502,1503,1504,1505,1506,1507,1508,1509,1510,1511,1512,1513,1514,1515,1516,1517,1518,1519,1520,1521,1522,1523,1524,1525,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535,1536,1537,1538,1539,1540,1541,1542,1543,1544,1545,1546,1547,1548,1549,1550,1551,1552,1553,1554,1555,1556,1557,1558,1559,1560,1561,1562,1563,1564,1565,1566,1567,1568,1569,1570,1571,1572,1573,1574,1575,1576,1577,1578,1579,1580,1581,1582,1583,1584,1585,1586,1587,1588,1589,1590,1591,1592,1593,1594,1595,1596,1597,1598,1599,1600,1601,1602,1603,1604,1605,1606,1607,1608,1609,1610,1611,1612,1613,1614,1615,1616,1617,1618,1619,1620,1621,1622,1623,1624,1625,1626,1627,1628,1629,1630,1631,1632,1633,1634,1635,1636,1637,1638,1639,1640,1641,1642,1643,1644,1645,1646,1647,1648,1649,1650,COUNT_DRUG_TYPE_0,COUNT_DRUG_TYPE_1,COUNT_DRUG_TYPE_2,COUNT_DRUG_TYPE_3,COUNT_DRUG_TYPE_4,COUNT_DRUG_TYPE_5,COUNT_DRUG_TYPE_6,COUNT_DRUG_TYPE_7,COUNT_DRUG_TYPE_8,COUNT_DRUG_TYPE_9,COUNT_DRUG_TYPE_10,COUNT_DRUG_TYPE_11,COUNT_DRUG_TYPE_12,COUNT_DRUG_TYPE_13,COUNT_DRUG_TYPE_14,COUNT_DRUG_TYPE_15,COUNT_DRUG_TYPE_16,COUNT_DRUG_TYPE_17,COUNT_DRUG_TYPE_18,COUNT_SYMPTOM_TYPE_0,COUNT_SYMPTOM_TYPE_1,COUNT_SYMPTOM_TYPE_2,COUNT_SYMPTOM_TYPE_3,COUNT_SYMPTOM_TYPE_4,COUNT_SYMPTOM_TYPE_5,COUNT_SYMPTOM_TYPE_6,COUNT_SYMPTOM_TYPE_7,COUNT_SYMPTOM_TYPE_8,COUNT_SYMPTOM_TYPE_9,COUNT_SYMPTOM_TYPE_10,COUNT_SYMPTOM_TYPE_11,COUNT_SYMPTOM_TYPE_12,COUNT_SYMPTOM_TYPE_13,COUNT_SYMPTOM_TYPE_14,COUNT_SYMPTOM_TYPE_15,COUNT_SYMPTOM_TYPE_16,COUNT_SYMPTOM_TYPE_17,COUNT_SYMPTOM_TYPE_18,COUNT_SYMPTOM_TYPE_19,COUNT_SYMPTOM_TYPE_20,COUNT_SYMPTOM_TYPE_21,COUNT_SYMPTOM_TYPE_22,COUNT_SYMPTOM_TYPE_23,COUNT_SYMPTOM_TYPE_24,COUNT_SYMPTOM_TYPE_25,COUNT_SYMPTOM_TYPE_26,COUNT_SYMPTOM_TYPE_27,COUNT_SYMPTOM_TYPE_28,COUNT_SYMPTOM_TYPE_29,COUNT_TEST_TYPE_0,COUNT_TEST_TYPE_1,COUNT_TEST_TYPE_2,COUNT_TEST_TYPE_3,COUNT_TEST_TYPE_4,COUNT_TEST_TYPE_5,COUNT_PRIMARY_DIAGNOSIS,days_since_last_incident_max,days_since_last_incident_mean,days_since_last_incident_std,days_since_last_incident_min,days_ofcourse_max,days_since_last_incident_DRUG_TYPE_0_std,days_since_last_incident_DRUG_TYPE_1_std,days_since_last_incident_DRUG_TYPE_2_std,days_since_last_incident_DRUG_TYPE_3_std,days_since_last_incident_DRUG_TYPE_4_std,days_since_last_incident_DRUG_TYPE_5_std,days_since_last_incident_DRUG_TYPE_6_std,days_since_last_incident_DRUG_TYPE_7_std,days_since_last_incident_DRUG_TYPE_8_std,days_since_last_incident_DRUG_TYPE_9_std,days_since_last_incident_DRUG_TYPE_10_std,days_since_last_incident_DRUG_TYPE_11_std,days_since_last_incident_DRUG_TYPE_12_std,days_since_last_incident_DRUG_TYPE_13_std,days_since_last_incident_DRUG_TYPE_14_std,days_since_last_incident_DRUG_TYPE_15_std,days_since_last_incident_DRUG_TYPE_16_std,days_since_last_incident_DRUG_TYPE_17_std,days_since_last_incident_DRUG_TYPE_18_std,days_since_last_incident_SYMPTOM_TYPE_0_std,days_since_last_incident_SYMPTOM_TYPE_1_std,days_since_last_incident_SYMPTOM_TYPE_2_std,days_since_last_incident_SYMPTOM_TYPE_3_std,days_since_last_incident_SYMPTOM_TYPE_4_std,days_since_last_incident_SYMPTOM_TYPE_5_std,days_since_last_incident_SYMPTOM_TYPE_6_std,days_since_last_incident_SYMPTOM_TYPE_7_std,days_since_last_incident_SYMPTOM_TYPE_8_std,days_since_last_incident_SYMPTOM_TYPE_9_std,days_since_last_incident_SYMPTOM_TYPE_10_std,days_since_last_incident_SYMPTOM_TYPE_11_std,days_since_last_incident_SYMPTOM_TYPE_12_std,days_since_last_incident_SYMPTOM_TYPE_13_std,days_since_last_incident_SYMPTOM_TYPE_14_std,days_since_last_incident_SYMPTOM_TYPE_15_std,days_since_last_incident_SYMPTOM_TYPE_16_std,days_since_last_incident_SYMPTOM_TYPE_17_std,days_since_last_incident_SYMPTOM_TYPE_18_std,...,days_since_last_incident_SYMPTOM_TYPE_6_min,days_since_last_incident_SYMPTOM_TYPE_7_min,days_since_last_incident_SYMPTOM_TYPE_8_min,days_since_last_incident_SYMPTOM_TYPE_9_min,days_since_last_incident_SYMPTOM_TYPE_10_min,days_since_last_incident_SYMPTOM_TYPE_11_min,days_since_last_incident_SYMPTOM_TYPE_12_min,days_since_last_incid

In [21]:
pred = []
for i, row in tqdm.tqdm(test_df.iterrows()):    
    preds = []
    for i in range(5):
        preds.append(models[i].predict_proba(np.array(row[features]).reshape(1, -1)))
    x = np.mean(preds, axis=0)
    threshold = 0.47
    a = (x[0][1] > threshold).astype(int)
    pred.append(a)

11482it [02:52, 66.53it/s]


In [22]:
test_df['label'] = pred

In [23]:
test_df

,Patient-Uid,1501,1502,1503,1504,1505,1506,1507,1508,1509,1510,1511,1512,1513,1514,1515,1516,1517,1518,1519,1520,1521,1522,1523,1524,1525,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535,1536,1537,1538,1539,1540,1541,1542,1543,1544,1545,1546,1547,1548,1549,1550,1551,1552,1553,1554,1555,1556,1557,1558,1559,1560,1561,1562,1563,1564,1565,1566,1567,1568,1569,1570,1571,1572,1573,1574,1575,1576,1577,1578,1579,1580,1581,1582,1583,1584,1585,1586,1587,1588,1589,1590,1591,1592,1593,1594,1595,1596,1597,1598,1599,1600,1601,1602,1603,1604,1605,1606,1607,1608,1609,1610,1611,1612,1613,1614,1615,1616,1617,1618,1619,1620,1621,1622,1623,1624,1625,1626,1627,1628,1629,1630,1631,1632,1633,1634,1635,1636,1637,1638,1639,1640,1641,1642,1643,1644,1645,1646,1647,1648,1649,1650,COUNT_DRUG_TYPE_0,COUNT_DRUG_TYPE_1,COUNT_DRUG_TYPE_2,COUNT_DRUG_TYPE_3,COUNT_DRUG_TYPE_4,COUNT_DRUG_TYPE_5,COUNT_DRUG_TYPE_6,COUNT_DRUG_TYPE_7,COUNT_DRUG_TYPE_8,COUNT_DRUG_TYPE_9,COUNT_DRUG_TYPE_10,COUNT_DRUG_TYPE_11,COUNT_DRUG_TYPE_12,COUNT_DRUG_TYPE_13,COUNT_DRUG_TYPE_14,COUNT_DRUG_TYPE_15,COUNT_DRUG_TYPE_16,COUNT_DRUG_TYPE_17,COUNT_DRUG_TYPE_18,COUNT_SYMPTOM_TYPE_0,COUNT_SYMPTOM_TYPE_1,COUNT_SYMPTOM_TYPE_2,COUNT_SYMPTOM_TYPE_3,COUNT_SYMPTOM_TYPE_4,COUNT_SYMPTOM_TYPE_5,COUNT_SYMPTOM_TYPE_6,COUNT_SYMPTOM_TYPE_7,COUNT_SYMPTOM_TYPE_8,COUNT_SYMPTOM_TYPE_9,COUNT_SYMPTOM_TYPE_10,COUNT_SYMPTOM_TYPE_11,COUNT_SYMPTOM_TYPE_12,COUNT_SYMPTOM_TYPE_13,COUNT_SYMPTOM_TYPE_14,COUNT_SYMPTOM_TYPE_15,COUNT_SYMPTOM_TYPE_16,COUNT_SYMPTOM_TYPE_17,COUNT_SYMPTOM_TYPE_18,COUNT_SYMPTOM_TYPE_19,COUNT_SYMPTOM_TYPE_20,COUNT_SYMPTOM_TYPE_21,COUNT_SYMPTOM_TYPE_22,COUNT_SYMPTOM_TYPE_23,COUNT_SYMPTOM_TYPE_24,COUNT_SYMPTOM_TYPE_25,COUNT_SYMPTOM_TYPE_26,COUNT_SYMPTOM_TYPE_27,COUNT_SYMPTOM_TYPE_28,COUNT_SYMPTOM_TYPE_29,COUNT_TEST_TYPE_0,COUNT_TEST_TYPE_1,COUNT_TEST_TYPE_2,COUNT_TEST_TYPE_3,COUNT_TEST_TYPE_4,COUNT_TEST_TYPE_5,COUNT_PRIMARY_DIAGNOSIS,days_since_last_incident_max,days_since_last_incident_mean,days_since_last_incident_std,days_since_last_incident_min,days_ofcourse_max,days_since_last_incident_DRUG_TYPE_0_std,days_since_last_incident_DRUG_TYPE_1_std,days_since_last_incident_DRUG_TYPE_2_std,days_since_last_incident_DRUG_TYPE_3_std,days_since_last_incident_DRUG_TYPE_4_std,days_since_last_incident_DRUG_TYPE_5_std,days_since_last_incident_DRUG_TYPE_6_std,days_since_last_incident_DRUG_TYPE_7_std,days_since_last_incident_DRUG_TYPE_8_std,days_since_last_incident_DRUG_TYPE_9_std,days_since_last_incident_DRUG_TYPE_10_std,days_since_last_incident_DRUG_TYPE_11_std,days_since_last_incident_DRUG_TYPE_12_std,days_since_last_incident_DRUG_TYPE_13_std,days_since_last_incident_DRUG_TYPE_14_std,days_since_last_incident_DRUG_TYPE_15_std,days_since_last_incident_DRUG_TYPE_16_std,days_since_last_incident_DRUG_TYPE_17_std,days_since_last_incident_DRUG_TYPE_18_std,days_since_last_incident_SYMPTOM_TYPE_0_std,days_since_last_incident_SYMPTOM_TYPE_1_std,days_since_last_incident_SYMPTOM_TYPE_2_std,days_since_last_incident_SYMPTOM_TYPE_3_std,days_since_last_incident_SYMPTOM_TYPE_4_std,days_since_last_incident_SYMPTOM_TYPE_5_std,days_since_last_incident_SYMPTOM_TYPE_6_std,days_since_last_incident_SYMPTOM_TYPE_7_std,days_since_last_incident_SYMPTOM_TYPE_8_std,days_since_last_incident_SYMPTOM_TYPE_9_std,days_since_last_incident_SYMPTOM_TYPE_10_std,days_since_last_incident_SYMPTOM_TYPE_11_std,days_since_last_incident_SYMPTOM_TYPE_12_std,days_since_last_incident_SYMPTOM_TYPE_13_std,days_since_last_incident_SYMPTOM_TYPE_14_std,days_since_last_incident_SYMPTOM_TYPE_15_std,days_since_last_incident_SYMPTOM_TYPE_16_std,days_since_last_incident_SYMPTOM_TYPE_17_std,days_since_last_incident_SYMPTOM_TYPE_18_std,...,days_since_last_incident_SYMPTOM_TYPE_7_min,days_since_last_incident_SYMPTOM_TYPE_8_min,days_since_last_incident_SYMPTOM_TYPE_9_min,days_since_last_incident_SYMPTOM_TYPE_10_min,days_since_last_incident_SYMPTOM_TYPE_11_min,days_since_last_incident_SYMPTOM_TYPE_12_min,days_since_last_incident_SYMPTOM_TYPE_13_min,days_since_last_inci

In [24]:
test_df[['Patient-Uid', 'label']].to_csv("./data/sample_submission.csv", index=False)